In [ ]:
import json
import redis
import paho.mqtt.client as mqtt

REDIS_HOST = ''
REDIS_PORT = 
REDIS_USERNAME = ''
REDIS_PASSWORD = ''

try:
    redis_client = redis.Redis(
        host=REDIS_HOST, 
        port=REDIS_PORT, 
        username=REDIS_USERNAME, 
        password=REDIS_PASSWORD
    )
    print("Redis Connected:", redis_client.ping())
except Exception as e:
    print("Redis Connection Error:", e)

# MQTT Configuration w,th defining the broker address also
BROKER_ADDRESS = "broker.emqx.io"
PORT = 1883
TOPIC = "s336966_s338142_s343124" 

# Define Callbacks
def on_connect(client, userdata, flags, rc):
    print(f"Connected to MQTT Broker with result code {rc}")
    client.subscribe(TOPIC)

def on_message(client, userdata, msg):
    try:
        # Parse the message 
        payload = msg.payload.decode()
        data_dict = json.loads(payload)
        
        mac_addr = data_dict['mac_address']
        timestamp = data_dict['timestamp']
        measurements = data_dict['data']
        
        print(f"Received data from {mac_addr} at {timestamp}")

        # Store in Redis database
        for item in measurements:
            # Construct key: <mac_address>:temperature or <mac_address>:humidity 
            key = f"{mac_addr}:{item['name']}"
            value = item['value']
            
            # Create key if it doesn't exist (
            try:
                redis_client.ts().create(key)
            except redis.ResponseError:
                pass # Key already exists
            
            # Add sample to TimeSeries
            redis_client.ts().add(key, timestamp, value)
            print(f"Stored {key}: {value}")
            
    except Exception as e:
        print(f"Error processing message: {e}")

# Run Subscriber
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(BROKER_ADDRESS, PORT)
client.loop_forever() # Blocks execution to listen for messages

Redis Connected: True
Connected to MQTT Broker with result code 0
/tmp/ipykernel_98/1093775819.py:65: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()
Received data from 26:99:64:91:45:14 at 1766154070291
Stored 26:99:64:91:45:14:temperature: 20
Stored 26:99:64:91:45:14:humidity: 58
Received data from 26:99:64:91:45:14 at 1766154075546
Stored 26:99:64:91:45:14:temperature: 20
Stored 26:99:64:91:45:14:humidity: 58
Received data from 26:99:64:91:45:14 at 1766154080803
Stored 26:99:64:91:45:14:temperature: 20
Stored 26:99:64:91:45:14:humidity: 58
Received data from 26:99:64:91:45:14 at 1766154086058
Stored 26:99:64:91:45:14:temperature: 20
Stored 26:99:64:91:45:14:humidity: 58
Received data from 26:99:64:91:45:14 at 1766154091315
Stored 26:99:64:91:45:14:temperature: 20
Stored 26:99:64:91:45:14:humidity: 57
Received data from 26:99:64:91:45:14 at 1766154096570
Stored 26:99:64:91:45:14:temperature: 20
Stored 26:99:64:91:45:14:humid

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=37068574-a210-48cc-b0b2-9175710228d6' target="_blank">

Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>